In [ ]:
import glob
import pathlib
import PIL
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
data_dir="/content/drive/MyDrive/pedestrian_dataset/"
data_dir=pathlib.Path(data_dir)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# One Time Code

In [ ]:
path=str(data_dir)
!mkdir {path}/yolov5

In [ ]:
path=path+"/yolov5"
!mkdir {path}/images
!mkdir {path}/labels

images_path=path+"/images"
labels_path=path+"/labels"

!mkdir {images_path}/train
!mkdir {images_path}/val
!mkdir {labels_path}/train
!mkdir {labels_path}/val

In [ ]:
images_path=path+"/images"
labels_path=path+"/labels"

images_path=pathlib.Path(images_path)
labels_path=pathlib.Path(labels_path)

In [ ]:

train_images=images_path.glob("train/*")
test_images=images_path.glob("val/*")

train_labels=labels_path.glob("train/*")
test_labels=labels_path.glob("val/*")

print(len(list(train_labels)))
print(len(list(train_images)))
print(len(list(test_images)))
print(len(list(test_labels)))

1810
1923
651
586


## YOLO DETECTION

In [ ]:
#clone YOLOv5 and 
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt # install dependencies
# %pip install -q roboflow

import torch
import os
from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Cloning into 'yolov5'...
remote: Enumerating objects: 12262, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 12262 (delta 2), reused 6 (delta 1), pack-reused 12250
Receiving objects: 100% (12262/12262), 11.96 MiB | 31.82 MiB/s, done.
Resolving deltas: 100% (8488/8488), done.
/content/yolov5
     |████████████████████████████████| 596 kB 30.3 MB/s 
Setup complete. Using torch 1.11.0+cu113 (Tesla T4)


In [ ]:
!cp /content/drive/MyDrive/pedestrian_dataset/yolov5/config/yolov5m.yaml /content/yolov5/models/yolov5m.yaml

In [ ]:
!cp /content/drive/MyDrive/pedestrian_dataset/yolov5/config/yolov5s.yaml /content/yolov5/models/yolov5s.yaml

In [ ]:
!cat /content/yolov5/models/yolov5s.yaml

# YOLOv5 🚀 by Ultralytics, GPL-3.0 license

# Parameters
nc: 2  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],

In [ ]:
!cp /content/drive/MyDrive/pedestrian_dataset/yolov5/config/pedestrian_car_data.yaml /content/yolov5/data/pedestrian_car_data.yaml

In [ ]:
!cat /content/yolov5/data/pedestrian_car_data.yaml

train: ../../content/drive/MyDrive/pedestrian_dataset/yolov5/images/train/ 
val:  ../../content/drive/MyDrive/pedestrian_dataset/yolov5/images/val/


# number of classes
nc: 2

# class names
names: ["car","ped"]


In [ ]:
!python train.py --img 640 --batch 32 --epochs 300 --data /content/yolov5/data/pedestrian_car_data.yaml --cfg /content/yolov5/models/yolov5s.yaml --weights yolov5s.pt --cache --project "../../content/drive/MyDrive/pedestrian_dataset/yolov5/weights2/" --name "backup2"

train: weights=yolov5s.pt, cfg=/content/yolov5/models/yolov5s.yaml, data=/content/yolov5/data/pedestrian_car_data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=500, batch_size=64, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=../../content/drive/MyDrive/pedestrian_dataset/yolov5/weights2/, name=backup2, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-258-g1156a32 Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05

In [ ]:
!python train.py --resume /content/drive/MyDrive/pedestrian_dataset/yolov5/weights4/backup2